# Figure panel 5

In [1]:
options(warn=-1)

In [2]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(tradeSeq),

            # CCI 
            library(CellChat), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [3]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [4]:
random_seed <- 42
set.seed(random_seed)

In [5]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [6]:
# Source
source("bin/so_pl.R")
source("bin/dea_pp.R")
source("bin/dea_pl.R")
source("bin/gsea_pp.R")
source("bin/gsea_pl.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")

In [7]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Import scRNAseq data 

In [8]:
so <- readRDS("data/scRNAseq/object/pp_1.rds")
so <- NormalizeData(so)

Normalizing layer: counts



# Differential expression analysis (DEA)

In [9]:
# Response gene DEA 
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_LysM_cre_Baseline_D0")
dea_res_2 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_3 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D3", "IFNAR_fl_LysM_cre_Baseline_D0")
dea_res_4 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

# Genotype DEA
contrasts_vec <- c("IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_Baseline_D0")
dea_res_5 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D1")
dea_res_6 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D3", "IFNAR_fl_CpG_D3")
dea_res_7 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [10]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.25

# DEG count overview

## Response count D1 and D3 vs Ctl 

In [11]:
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_2 <- lapply(dea_res_2, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_3 <- lapply(dea_res_3, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_4 <- lapply(dea_res_4, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 

In [12]:
mat <- list(mat_1, mat_2, mat_3, mat_4)

In [13]:
mat <- lapply(mat, function(x) {
    
    x <- as.data.frame(x)
    set_diff <- setdiff(make.names(levels(so$celltype_low)), rownames(x))
    x[set_diff, ] <- 0
    x[make.names(levels(so$celltype_low)), , drop=FALSE]

}
             )

mat <- do.call(cbind, mat)

In [14]:
colnames(mat) <- c("+/+", "cre/+", "+/+", "cre/+")
col_split <- c("D1", "D1", "D3", "D3")

In [15]:
pdf("result/figures/1_scRNAseq/panel_5/hm_deg_count_d1_d3_vs_ctl_cg_and_ko.pdf", width=1.5, height=3)
source("bin/dea_pl.R")
draw(dea_count_hm(as.matrix(mat), col_split=col_split, fontsize_select=2))

dev.off()

pdf 
  2

## Genotype comparison 

In [16]:
mat_1 <- lapply(dea_res_5, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_2 <- lapply(dea_res_6, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_3 <- lapply(dea_res_7, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 

In [17]:
mat <- list(mat_1, mat_2, mat_3)

In [18]:
mat <- lapply(mat, function(x) {
    
    x <- as.data.frame(x)
    set_diff <- setdiff(make.names(levels(so$celltype_low)), rownames(x))
    x[set_diff, ] <- 0
    x[make.names(levels(so$celltype_low)), , drop=FALSE]

}
             )

mat <- do.call(cbind, mat)

In [19]:
colnames(mat) <- c("+/+ vs cre/+", "+/+ vs cre/+", "+/+ vs cre/+")
col_split <- c("Ctl", "D1", "D3")

In [20]:
pdf("result/figures/1_scRNAseq/panel_5/hm_deg_count_cg_vs_ko.pdf", width=1.5, height=3)

draw(dea_count_hm(as.matrix(mat), col_split=col_split, fontsize_select=2))

dev.off()

pdf 
  2

# Volcano plots  

## CtlKO vs Ctl

In [21]:
labels <- list(
    
    "intMo"=c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Bach1", "Il1a", "Il1b", "Il1rn", "Il18", "Il33")

)

In [22]:
vp_5 <- lapply(names(dea_res_5), function(i) {

    v_pl_i <- v_pl(dea_res_5[[i]], title=i, log2FC_thr=log2FC_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_LysM_cre_Baseline_D0"]), color_neg=unlist(color$sample_group["IFNAR_fl_Baseline_D0"]), label=if(i %in% names(labels)) {labels[[i]]} else {label <- NULL}, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5, label_merge=TRUE) + theme(legend.position="none") + theme_global_set(size_select=4)

}
                 )

In [23]:
vp_5 <- lapply(vp_5, function(p) {egg::set_panel_size(p, width=unit(3, "cm"), height=unit(2.5, "cm"))})
vp_5 <- do.call(gridExtra::arrangeGrob, c(vp_5, ncol=5, nrow=ceiling(length(vp_5)/5)))

In [24]:
pdf("result/figures/1_scRNAseq/panel_5/vp_ctlko_vs_ctl.pdf", width=2*5, height=2*ceiling(length(vp_5)/5))

grid::grid.draw(vp_5)

dev.off()

pdf 
  2

## D1KO vs D1

In [25]:
labels <- list(
    
    "intMo"=c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Bach1", "Il1a", "Il1b", "Il1rn", "Il18", "Il33")

)

In [26]:
vp_6 <- lapply(names(dea_res_6), function(i) {

    v_pl_i <- v_pl(dea_res_6[[i]], title=i, log2FC_thr=log2FC_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"]), color_neg=unlist(color$sample_group["IFNAR_fl_CpG_D1"]), label=if(i %in% names(labels)) {labels[[i]]} else {label <- NULL}, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5, label_merge=TRUE) + theme(legend.position="none") + theme_global_set(size_select=4)

}
                 )

In [27]:
vp_6 <- lapply(vp_6, function(p) {egg::set_panel_size(p, width=unit(3, "cm"), height=unit(2.5, "cm"))})
vp_6 <- do.call(gridExtra::arrangeGrob, c(vp_6, ncol=5, nrow=ceiling(length(vp_6)/5)))

In [28]:
pdf("result/figures/1_scRNAseq/panel_5/vp_d1ko_vs_d1.pdf", width=2*5, height=2*ceiling(length(vp_6)/5))

grid::grid.draw(vp_6)

dev.off()

pdf 
  2

## D3KO vs D3

In [29]:
labels <- list(
    
    "intMo"=c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Bach1", "Il1a", "Il1b", "Il1rn", "Il18", "Il33")

)

In [30]:
vp_7 <- lapply(names(dea_res_7), function(i) {

    v_pl_i <- v_pl(dea_res_7[[i]], title=i, log2FC_thr=log2FC_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), label=if(i %in% names(labels)) {labels[[i]]} else {label <- NULL}, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5, label_merge=TRUE) + theme(legend.position="none") + theme_global_set(size_select=4)

}
                 )

In [31]:
vp_7 <- lapply(vp_7, function(p) {egg::set_panel_size(p, width=unit(3, "cm"), height=unit(2.5, "cm"))})
vp_7 <- do.call(gridExtra::arrangeGrob, c(vp_7, ncol=5, nrow=ceiling(length(vp_7)/5)))

In [32]:
pdf("result/figures/1_scRNAseq/panel_5/vp_d3ko_vs_d3.pdf", width=2*5, height=2*ceiling(length(vp_7)/5))

grid::grid.draw(vp_7)

dev.off()

pdf 
  2

# Heat map log2FC

In [33]:
perturbed_genes_1 <- dea_res_6[["intMo"]] %>% dplyr::filter(avg_log2FC<=-0.25 & p_val_adj<=0.05) %>% pull(gene)
perturbed_genes_2 <- dea_res_7[["intMo"]] %>% dplyr::filter(avg_log2FC<=-0.25 & p_val_adj<=0.05) %>% pull(gene)
perturbed_genes_3 <- dea_res_6[["cMo"]] %>% dplyr::filter(avg_log2FC<=-0.25 & p_val_adj<=0.05) %>% pull(gene)
perturbed_genes_4 <- dea_res_7[["cMo"]] %>% dplyr::filter(avg_log2FC<=-0.25 & p_val_adj<=0.05) %>% pull(gene)

In [34]:
diff_genes <- c(perturbed_genes_1, perturbed_genes_2, perturbed_genes_3, perturbed_genes_4) %>% unique()
length(diff_genes)

[1] 1536

In [35]:
col_samples <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3")
col_split <- c("Ctl", "Ctl", "D1", "D1", "D3", "D3")
col_label <- c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")

In [36]:
row_genes <- list(

    "G1"=c("Ccr2", "Cd163", "Adgre1", "Spic", "Slfn4"), 
    
    "G2"=c("Gdf15", "Ccl2", "Ccl12", "Cxcl10", "Il1b", "Il18bp", "Ccl4", "Cxcl2", "Tnf"), # Inflamasome Il1b, Il18
    
    "G3"=c("Slc48a1", "Ctsb", "Ctsd", "Slc40a1", "Ftl1", "Fth1", "Hmox1", "Blvra"), # Slc48a1 (Hrg1), Nfe2l2 is constitutively expressed in intMo
    
    "G4"=c("Scarf1",  "Tmem41b", "Vps11"),
    
    "G5"=c("Stat1", "Stat2", "Irf1", "Irf9", "Irf7", "Ifih1", "Isg15", "Isg20", "Ifi44", "Ifit1", "Ifit2", "Ifit3", "Ifit3b", "Ifitm1", "Ifitm3", "Usp18", "Oasl1", "Oasl2", "Ddx60", "Cd274", "Bst2"), 
    
    "G6"=c("H2-Aa", "H2-Ab1", "H2-Eb1", "Cd74", "Ciita"), 
    
    "G7"=c("Gbp2", "Gbp2b", "Gbp3", "Gbp4", "Gbp5", "Gbp6", "Gbp7", "Gbp8"), 
    
    "G9"=c("Maf", "Mafb", "Trib1", "Egr1", "Fos", "Fosb", "Jun"), 

    "G10"=c("Lcn2"), 
    
    "G11"=c("Bcl2", "Bcl2l1", "Bcl2l11"), # Cre LysM Bim fl/fl

    "G12"=c("Atf4", "Gdf15", "Eif2ak2")
)

In [37]:
row_genes <- list("KO UP"=diff_genes)

In [38]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
row_split[!row_split %in% rownames(so)]

named character(0)

In [39]:
hm_dea_fetch <- function(celltype, scale_mode="all") {
    
    # Get pseudobulk counts 
    so_i <- so[, (so$celltype_low %in% celltype_select) & (so$sample_group %in% col_samples), drop=FALSE]

    # Get pseudobulk counts 
    mat_1 <- GetAssayData(so_i, assay="RNA", layer="counts")
    mat_1 <- split(t(mat_1) %>% as.data.frame(), f=paste(so_i$celltype_low, so_i$sample_group))
    mat_1 <- lapply(mat_1, colSums) %>% do.call(rbind, .) %>% t() 
    mat_1 <- t(t(mat_1) / colSums(mat_1))
    
    if(scale_mode=="all") {

        mat_1 <- apply(mat_1, 1, scales::rescale) %>% t() # It apply transforms the data frame
        mat_1 <- mat_1[row_split, paste(celltype, col_samples)]

        
    } else if (scale_mode=="sample") {

        mat_1 <- mat_1[row_split, paste(celltype, col_samples)]
        mat_1 <- apply(mat_1, 1, scales::rescale) %>% t() # It apply transforms the data frame
        
    }

    colnames(mat_1) <- col_label

    row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
    row_split <- row_split[row_split %in% rownames(mat_1)]

    # DEA log2FC 
    mat_2 <- purrr::reduce(
        
        list(
            
            dea_res_5[[celltype]] %>% dplyr::mutate(Ctl=avg_log2FC) %>% dplyr::select(gene, Ctl), 
            dea_res_6[[celltype]] %>% dplyr::mutate(D1=avg_log2FC) %>% dplyr::select(gene, D1), 
            dea_res_7[[celltype]] %>% dplyr::mutate(D3=avg_log2FC) %>% dplyr::select(gene, D3)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_2[setdiff(row_split, rownames(mat_2)), ] <- 0
    mat_2 <- mat_2[row_split, ]
    mat_2[is.na(mat_2)] <- 0
    
    # DEA adj p-values
    mat_3 <- purrr::reduce(
        
        list(
            
            dea_res_5[[celltype]] %>% dplyr::mutate(Ctl=p_val_adj) %>% dplyr::select(gene, Ctl), 
            dea_res_6[[celltype]] %>% dplyr::mutate(D1=p_val_adj) %>% dplyr::select(gene, D1), 
            dea_res_7[[celltype]] %>% dplyr::mutate(D3=p_val_adj) %>% dplyr::select(gene, D3)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_3[setdiff(row_split, rownames(mat_3)), ] <- 1
    mat_3 <- mat_3[row_split, ]
    mat_3[is.na(mat_3)] <- 1
    mat_3[(mat_3>adj_pval_thr | abs(mat_2)<log2FC_thr)] <- 1

    # Row split
    row_split=names(row_split)
    row_split=factor(row_split, levels=names(row_genes))
    
    hm_1 <- dea_exp_hm(mat_1, col_split=col_split, row_split=row_split, scale_width=1, legend_title="Exp.", cluster_rows=TRUE, fontsize_select=2)
    hm_2 <- dea_res_hm(mat_2, mat_3, p_val_adj_thr=adj_pval_thr, row_split=row_split, scale_width=1, color_min="#cd34b5", color_max="#FFAC1E", cluster_rows=TRUE, fontsize_select=2)

    hm <- hm_2 + hm_1

    return(hm)
}

In [40]:
celltype_select <- c("cMo", "intMo", "RPM")

In [41]:
hm_1 <- hm_dea_fetch(celltype="cMo")
hm_2 <- hm_dea_fetch(celltype="intMo")
hm_3 <- hm_dea_fetch(celltype="RPM")

In [42]:
pdf("result/figures/1_scRNAseq/panel_5/hm_dea_ctl_d1_d3_genotype_ko_down.pdf", width=5, height=200)

hm_1 + hm_2 + hm_3

dev.off()

pdf 
  2

# Gene set enrichment analysis (GSEA)

## Genotype comparison - M7

In [316]:
compute_gsea <- FALSE
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC1", "cDC2", "moDC", "cDC.mig.", "Basophil")

In [ ]:
if(compute_gsea) {

    # Collapse gene set
    gene_set <- msigdbr::msigdbr(species="mouse", db_species="MM", category="M7", subcategory=NULL)
    gene_set <- gene_set[grep("_UP", gene_set$gs_name), ]
    gene_set$gs_name <- sapply(strsplit(gene_set$gs_name, "_"), function(x) {paste(x[max(1, length(x)-3):(length(x)-2)], collapse="_")})
    gene_set$gs_name <- gsub("CELL_|CD4_|CD8_|MONOCYTE_|MACROPHAGE_|TREG_|LANGERHANS_|NEUTROPHIL_|CDC1_|CDC2_|MIGDC_|ETAC_|GD_|ILC_|PDC_", "", gene_set$gs_name)
    gene_set <- gene_set %>% dplyr::select(gene_symbol, gs_name) %>% dplyr::distinct()

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea(dea_res_5[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea(dea_res_6[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea(dea_res_7[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_m7.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_m7.rds")
    
}

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
row_split <- read.csv("data/reference/immune_dictionary/immune_dictionary.csv") %>% tibble::column_to_rownames("gs_name")
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
row_split <- row_split[rownames(mat_1), ]

In [ ]:
rownames(mat_1) <- row_split$norm
rownames(mat_2) <- row_split$norm

In [ ]:
row_split <- row_split %>% dplyr::select(norm, family) %>% dplyr::rename(row_split_label=family) %>% dplyr::distinct()
rownames(row_split) <- row_split$norm

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_gsea_ctl_d1_d3_wt_vs_ko_m7.pdf", width=8, height=12)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=row_split, fontsize_select=2)

dev.off()

## Genotype comparison - HM

In [ ]:
compute_gsea <- FALSE
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC1", "cDC2", "moDC", "cDC.mig.", "Basophil")

In [ ]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea(dea_res_5[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea(dea_res_6[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea(dea_res_7[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_mh.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_mh.rds")
    
}

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gsea_res <- gsea_res[gsea_res$col_label %in% c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "RPM"), ]

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)
gs_name_select <- gs_name_select[grep("E2F|MYC|HYPOXIA|STAT3|INTERFERON|TNF|HEME", gs_name_select)]
gsea_res <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select)

In [ ]:
gs_name_order <- c("HALLMARK_INTERFERON_ALPHA_RESPONSE", "HALLMARK_INTERFERON_GAMMA_RESPONSE", "HALLMARK_IL6_JAK_STAT3_SIGNALING", "HALLMARK_TNFA_SIGNALING_VIA_NFKB", "HALLMARK_HYPOXIA", "HALLMARK_HEME_METABOLISM", "HALLMARK_MYC_TARGETS_V1", "HALLMARK_MYC_TARGETS_V2", "HALLMARK_E2F_TARGETS")
gsea_res$gs_name <- factor(gsea_res$gs_name, levels=gs_name_order)

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
mat_1 <- mat_1[gs_name_order, ]
mat_2 <- mat_2[gs_name_order, ]

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_gsea_ctl_d1_d3_wt_vs_ko_mh.pdf", width=8, height=10)

source("bin/gsea_pl.R")
gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="HALLMARK", color_neg=color$sample_group["IFNAR_fl_CpG_D1"], color_pos=color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"], fontsize_select=2)

dev.off()

## Genotype comparison - GO

In [ ]:
compute_gsea <- FALSE
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC1", "cDC2", "moDC", "cDC.mig.", "Basophil")

In [ ]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea_cp(dea_res_5[[i]]) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea_cp(dea_res_6[[i]]) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea_cp(dea_res_7[[i]]) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
}

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(p.adjust<=0.05 & abs(NES)>=1) %>% dplyr::pull(ID)
gsea_res <- gsea_res %>% dplyr::filter(ID %in% gs_name_select)

In [ ]:
gsea_res <- gsea_res[!gsea_res$Description %in% grep("T cell|B cell|CD4|CD8|lymphocyte|endothelial|epithelial", gsea_res$Description, value=TRUE), ]

In [ ]:
if(compute_gsea) {

    gs_reduce <- gsea_cp_reduce(gsea_res)

    saveRDS(gs_reduce, "result/gsea/scRNAseq/gsea_reduce_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
} else {

    gs_reduce <- readRDS("result/gsea/scRNAseq/gsea_reduce_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
}

In [ ]:
gsea_res <- gsea_res %>% dplyr::rename(pathway=Description, padj=p.adjust) %>% dplyr::filter(ID %in% gs_reduce$go)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=2) %>% dplyr::pull(ID)
gsea_res <- gsea_res %>% dplyr::filter(ID %in% gs_name_select)

In [ ]:
mat_1 <- gsea_res %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
row_split <- gs_reduce %>% dplyr::select(term, parentTerm) %>% dplyr::rename(row_split_label=parentTerm) %>% dplyr::distinct()
rownames(row_split) <- gs_reduce$term

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/hm_gsea_ctl_d1_d3_wt_vs_ko_gobp.pdf", width=10, height=50)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=row_split, pathway_suffix="WP", fontsize_select=2)

dev.off()

## Violine gene expression plots

In [ ]:
gene_select <- c("Ly6c2", "Spic", "Cd163", "Gdf15", "Ccl2", "Slfn4", "Bcl2l1", "Hmox1", "Jun")
celltype_select <- c("cMo", "intMo", "RPM")

In [ ]:
mat <- cbind(so@meta.data[, c("sample_group", "celltype_low"), drop=FALSE], t(GetAssayData(so, assay="RNA", slot="data")[gene_select, ])) %>% 
    pivot_longer(cols=-c(sample_group, celltype_low), names_to="gene", values_to="expression")

In [ ]:
map <- data.frame(
    
    sample_group=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3"),
    time_point=c("Ctl", "Ctl", "D1", "D1", "D3", "D3"),
    genotype=c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")
)

mat <- mat %>% dplyr::left_join(., map, by=join_by(sample_group)) %>% 
    dplyr::mutate(gene=factor(gene, levels=gene_select))

In [ ]:
genotype <- c("#732E8F", "#D390BF", "#FA8775")
names(genotype) <- c("WT", "+/+", "cre/+")

In [ ]:
options(repr.plot.width=5, repr.plot.height=7.5)

exp_p <- ggplot(mat %>% dplyr::filter(celltype_low %in% celltype_select), aes(x=time_point, y=expression, color=sample_group, fill=sample_group)) + 

        geom_jitter(position=position_jitterdodge(jitter.width=0.20, dodge.width=1.0), shape=21, stroke=0.1, size=0.75, show.legend=FALSE, color="black") + 
        # geom_boxplot(position=position_dodge(width=1.0), width=0.20, color="black", alpha=1, outlier.color=NA) + 
        
        xlab("") + ylab("log(CPTT)") + 
        ggtitle("Spic") + 
        facet_grid(gene~celltype_low, scales="free") + 
        scale_color_manual(values=color$sample_group) + 
        scale_fill_manual(values=color$sample_group)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/exp_myeloid.pdf", width=3, height=8)

set_height <- 1 * length(gene_select)
set_width <- 3 * length(celltype_select)

exp_p + ggh4x::force_panelsizes(cols=unit(set_width, "mm"), total_height=unit(set_height, "cm"))

dev.off()

# CCI DEA 

In [ ]:
# Load source 
source("bin/cci_pp.R")
source("bin/cci_pl.R")

# Parameter 
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.05
pathway_name_thr <- c("ICAM", "VCAM", "SIRP", "KIT", "TGFb", "TNF", "CXCL", "CCL")

# CellChat results
lr_res <- read.csv("result/cci/lr_res.csv", row.names=1)

# Add reversed Cell-cell interactions 
db <- CellChat::CellChatDB.mouse

# Impute custom interactions
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1, na.strings=c(""))
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

# Get cell-cell contact interaction 
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact"), key="annotation")

# Subset lr results for cell-cell contact forward 
lr_res_ccc_fwd <- lr_res %>% dplyr::filter(lr_res$interaction_name %in% db$interaction$interaction_name)

# Reverse cell-cell contact 
lr_res_ccc_rev <- data.frame(

    source=lr_res_ccc_fwd$target, 
    target=lr_res_ccc_fwd$source, 
    interaction_name=paste0(lr_res_ccc_fwd$receptor_symbol, "_", lr_res_ccc_fwd$ligand_symbol), 
    prob=lr_res_ccc_fwd$prob, 
    pval=lr_res_ccc_fwd$pval, 
    pathway_name=lr_res_ccc_fwd$pathway_name, 
    ligand_symbol=lr_res_ccc_fwd$receptor_symbol, 
    receptor_symbol=lr_res_ccc_fwd$ligand_symbol, 
    sample_group=lr_res_ccc_fwd$sample_group
    
)

# Combine results 
lr_res_ccc <- rbind(lr_res_ccc_fwd, lr_res_ccc_rev)

# Assemble LR results 
lr_res <- rbind(lr_res[!lr_res$interaction_name %in% lr_res_ccc$interaction_name, ], lr_res_ccc)

# Subset by interaction pathway
lr_res <- lr_res[lr_res$pathway_name %in% pathway_name_thr, ]
lr_res$pathway_name <- factor(lr_res$pathway_name, levels=pathway_name_thr)

# DEA results 

In [ ]:
dea_mat_5 <- dea_res_5[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_5 <- lapply(dea_mat_5, function(x) {list("Ctl"=x)})

In [ ]:
dea_mat_6<- dea_res_6[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_6 <- lapply(dea_mat_6, function(x) {list("D1"=x)})

In [ ]:
dea_mat_7 <- dea_res_7[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_7 <- lapply(dea_mat_7, function(x) {list("D3"=x)})

In [ ]:
dea_mat <- Map(c, dea_mat_5, dea_mat_6, dea_mat_7)

## PTPG B (+/+) vs D1 (+/+) and D3 (+/+)

## cMo

In [ ]:
source <- c("cMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_cmo_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_cmo_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_cmo_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## intMo

In [ ]:
source <- c("intMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_intmo_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_intmo_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_intmo_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## RPM

In [ ]:
source <- c("RPM")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_rpm_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_rpm_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cci_rpm_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

# Interaction plots

In [ ]:
sources <- c("RPM", "intMo", "cMo")
targets <- c("MEP", "Proerythroblast", "Erythroblast")

In [ ]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")
cco_4 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")
cco_5 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")
cco_6 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")

In [ ]:
cco_1 <- subsetCellChat(object=cco_1, idents.use=c(sources, targets))
cco_2 <- subsetCellChat(object=cco_2, idents.use=c(sources, targets))
cco_3 <- subsetCellChat(object=cco_3, idents.use=c(sources, targets))
cco_4 <- subsetCellChat(object=cco_4, idents.use=c(sources, targets))
cco_5 <- subsetCellChat(object=cco_5, idents.use=c(sources, targets))
cco_6 <- subsetCellChat(object=cco_6, idents.use=c(sources, targets))

In [ ]:
levels(cco_1@idents) <- c(targets, sources)
levels(cco_2@idents) <- c(targets, sources)
levels(cco_3@idents) <- c(targets, sources)
levels(cco_4@idents) <- c(targets, sources)
levels(cco_5@idents) <- c(targets, sources)
levels(cco_6@idents) <- c(targets, sources)

In [ ]:
color_use <- color$celltype_low[names(color$celltype_low) %in% c(sources, targets)]

In [ ]:
cco_list <- list(cco_1, cco_2, cco_3, cco_4, cco_5, cco_6)

q_prob_thr <- quantile(c(cco_1@net$prob, cco_2@net$prob, cco_3@net$prob), 0.90)
for (i in seq_along(cco_list)) {
    
    prob_arr <- cco_list[[i]]@net$prob
    prob_arr[prob_arr > q_prob_thr] <- q_prob_thr
    cco_list[[i]]@net$prob <- prob_arr
    
}

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cp_cci_net_ctl_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[4]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cp_cci_net_d1_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[5]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/cp_cci_net_d3_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[6]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

# Feature plots

In [ ]:
pdf("result/figures/1_scRNAseq/panel_5/fp_selected_genes.pdf", width=2*2.5, height=5*2.5)

gridExtra::grid.arrange(
    
    fplot(so, features="Ly6c2", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Ly6c2", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Spic", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Spic", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Vcam1", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Vcam1", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Gdf15", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Gdf15", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Ccl2", slot="data", restrict=c("sample_group", "IFNAR_fl_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
    fplot(so, features="Ccl2", slot="data", restrict=c("sample_group", "IFNAR_fl_LysM_cre_CpG_D1"), pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), ncol=2
    
)

dev.off()